In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-xrxnp52k
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-xrxnp52k
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
pip install konlpy

In [3]:
pip install keybert

In [4]:
!pip install rake-nltk

In [6]:
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from pykospacing import Spacing
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
from keybert import KeyBERT
import plotly.express as px
from rake_nltk import Rake
import networkx as nx
import plotly.graph_objects as go





nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# 데이터전처리

In [7]:
## 데이터셋 불러와서 합치기
df1 = pd.read_csv('/content/drive/MyDrive/직무기술서_사업관리.csv', encoding='utf-8')
df2 = pd.read_csv('/content/drive/MyDrive/직무기술서_전자기기.csv', encoding='utf-8')
df1['직무'] = '사업관리'
df2['직무'] = '전자기기'
df = pd.concat([df1, df2], ignore_index=True)
df

,□ 직무 기본정보,Unnamed: 1,Unnamed: 2,Unnamed: 3,직무
0,NaN,NaN,NaN,NaN,사업관리
1,직 무,공적개발원조사업관리,능력단위분류번호,0101010101_17v2,사업관리
2,NaN,NaN,능 력 단 위,공적개발원조사업 개발전략수립,사업관리
3,직무 목적,"공적개발원조사업 개발전략 수립이란 협력대상국의 정책, 제도, 법령, 현지여건을 토대...",NaN,NaN,사업관리
4,개발날짜\n(개선날짜),2017.10.27,개발기관\n(개선기관),한국소프트웨어산업협회,사업관리
...,...,...,...,...,...
7683,NaN,• 지속적으로 고객 정보를 수집하려는 자세,NaN,NaN,전자기기
7684,NaN,• 최근 동향을 파악하려는 자세,NaN,NaN,전자기기
7685,관련자격사항,• 소비자정보상담사\n• 텔레마케팅전문관리사\n• 경영지도사(마케팅),NaN,NaN,전자기기
7686,사전직무경험,• 전자제품 배송관리,NaN,NaN,전자기기


In [8]:
## 필요없는 3,4번째 column 제외
df = df.iloc[:, [0, 1, 4]]
## NaN 값들만 있는 row들 제외
df = df.dropna(how='all')
## 상세내용을 지식,기술,태도 카테고리에 매칭 시킬수 있도록 ffill 을 사용해 NaN 값들을 대체
df = df.fillna(method="ffill")
df.columns = ["카테고리","원본","직무"]
df

,카테고리,원본,직무
0,NaN,NaN,사업관리
1,직 무,공적개발원조사업관리,사업관리
2,직 무,공적개발원조사업관리,사업관리
3,직무 목적,"공적개발원조사업 개발전략 수립이란 협력대상국의 정책, 제도, 법령, 현지여건을 토대...",사업관리
4,개발날짜\n(개선날짜),2017.10.27,사업관리
...,...,...,...
7683,태도,• 지속적으로 고객 정보를 수집하려는 자세,전자기기
7684,태도,• 최근 동향을 파악하려는 자세,전자기기
7685,관련자격사항,• 소비자정보상담사\n• 텔레마케팅전문관리사\n• 경영지도사(마케팅),전자기기
7686,사전직무경험,• 전자제품 배송관리,전자기기


In [9]:
# 특정 컬럼에서 '기술', '태도', '지식'행만 필터링후 결과 확인
df = df[df["카테고리"].isin(["기술", "태도", "지식"])]
print(df["카테고리"].value_counts())
df

카테고리
지식    1531
기술    1456
태도    1228
Name: count, dtype: int64


,카테고리,원본,직무
22,지식,"• 국제기구, 양자원조기구, NGO 등의 협력대상국에 대한 개발정책과 지원현황",사업관리
23,지식,"• 자국의 공적개발원조 수행방식, 수행기관 업무",사업관리
24,지식,"• 자국의 공적개발원조 예산, 통계",사업관리
25,지식,• 자국의 공적개발원조 주요 지원 분야,사업관리
26,지식,"• 자국의 대외정책, 공적개발원조정책, 국별협력전략",사업관리
...,...,...,...
7680,태도,• 유관부서와 정보를 공유하려는 자세,전자기기
7681,태도,• 자사제품의 특장점을 설명하려는 자세,전자기기
7682,태도,• 제품을 상세히 설명하려는 자세,전자기기
7683,태도,• 지속적으로 고객 정보를 수집하려는 자세,전자기기


In [10]:
# 특수문자 제거하고 우선 토큰화 해서 확인
def remove_special_chars(text):
    # 한글, 숫자, 공백만 남기고 특수문자 제거
    cleaned_text = re.sub(r'[^가-힣0-9\s]', '', text)
    return cleaned_text

In [11]:
def tokenize_text(text):
    # 특수문자 제거
    cleaned_text = remove_special_chars(text)

    #토큰화
    tokens = word_tokenize(cleaned_text)

    return tokens

df['토큰화'] = df['원본'].apply(tokenize_text)
df

<ipython-input-11-b6a2a722679b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['토큰화'] = df['원본'].apply(tokenize_text)


,카테고리,원본,직무,토큰화
22,지식,"• 국제기구, 양자원조기구, NGO 등의 협력대상국에 대한 개발정책과 지원현황",사업관리,"[국제기구, 양자원조기구, 등의, 협력대상국에, 대한, 개발정책과, 지원현황]"
23,지식,"• 자국의 공적개발원조 수행방식, 수행기관 업무",사업관리,"[자국의, 공적개발원조, 수행방식, 수행기관, 업무]"
24,지식,"• 자국의 공적개발원조 예산, 통계",사업관리,"[자국의, 공적개발원조, 예산, 통계]"
25,지식,• 자국의 공적개발원조 주요 지원 분야,사업관리,"[자국의, 공적개발원조, 주요, 지원, 분야]"
26,지식,"• 자국의 대외정책, 공적개발원조정책, 국별협력전략",사업관리,"[자국의, 대외정책, 공적개발원조정책, 국별협력전략]"
...,...,...,...,...
7680,태도,• 유관부서와 정보를 공유하려는 자세,전자기기,"[유관부서와, 정보를, 공유하려는, 자세]"
7681,태도,• 자사제품의 특장점을 설명하려는 자세,전자기기,"[자사제품의, 특장점을, 설명하려는, 자세]"
7682,태도,• 제품을 상세히 설명하려는 자세,전자기기,"[제품을, 상세히, 설명하려는, 자세]"
7683,태도,• 지속적으로 고객 정보를 수집하려는 자세,전자기기,"[지속적으로, 고객, 정보를, 수집하려는, 자세]"


In [12]:
## 각 카테고리마다 어떠한 단어들이 많이 쓰이는지 확인
def get_top_10_words(df):
    top_words = {}
    for category in df['카테고리'].unique():
        category_words = df[df['카테고리'] == category]['토큰화'].explode()
        word_counts = Counter(category_words)
        top_words[category] = word_counts.most_common(15)  # 상위 10개 단어 추출
    return top_words

# 상위 10개 단어 계산
top_words = get_top_10_words(df)

# 결과 출력
for category, freq in top_words.items():
    print(f"카테고리: {category}")
    for word, count in freq:
        print(f"  {word}: {count}")
    print("-" * 30)

카테고리: 지식
  지식: 906
  대한: 887
  및: 189
  방법: 93
  이해: 84
  프로젝트: 65
  관한: 51
  관련: 48
  관리: 36
  전자제품: 36
  절차: 35
  방법에: 35
  위한: 30
  정보: 28
  국가별: 24
------------------------------
카테고리: 기술
  능력: 1140
  수: 217
  있는: 216
  기술: 210
  및: 176
  분석: 142
  대한: 114
  작성: 100
  파악: 70
  활용: 64
  관리: 61
  프로젝트: 46
  수립: 37
  위한: 34
  관련: 24
------------------------------
카테고리: 태도
  자세: 567
  태도: 461
  적극적으로: 90
  의지: 83
  적극적인: 80
  분석하려는: 80
  및: 79
  대한: 68
  하는: 63
  파악하려는: 63
  위한: 60
  정보를: 54
  노력: 53
  객관적으로: 51
  정확하게: 46
------------------------------


## 통계 분석 방법 vs 통계분석방법, 데이터 가공기술 vs 데이터 가공 기술 같은 데이터가 있어서 띄어쓰기 교정을 통해 데이터의 일관성을 확보 <br> 불필요하다고 생각하는 단어는 제거 ("지식","대한","및","능력","수","있는,"기술","자세","태도")하고  "분석하려는", "적극적으로", "파악하려는" 등의 단어는 "분석", "적극적", "파악" 이런식으로 변경

In [13]:
spacing = Spacing()
remove_words = {"지식","방법","자세", "대한","및","능력","수","있는","기술","자세","태도"}  # 제거할 단어 리스트


def tokenize_and_correct(text):
    # 특수문자 제거
    cleaned_text = remove_special_chars(text)

    # 띄어쓰기 교정
    corrected_text = spacing(cleaned_text)

    # 한글 토큰화
    tokens = word_tokenize(corrected_text)

    # 특정 단어 제거
    tokens = [word for word in tokens if word not in remove_words]


    # 특정 패턴("으로", "하려는") 제거
    tokens = [re.sub(r"(으로|하려는|하는)$", "", word) for word in tokens]

    # 단어 개수 계산
    word_count = len(tokens)

    result_text = " ".join(tokens)

    return result_text, word_count

# 데이터프레임에 두 개의 컬럼 추가
df[['전처리', '단어수']] = df['원본'].apply(lambda x: pd.Series(tokenize_and_correct(x)))
df


<ipython-input-13-55ba7fd2d37a>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['전처리', '단어수']] = df['원본'].apply(lambda x: pd.Series(tokenize_and_correct(x)))
<ipython-input-13-55ba7fd2d37a>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['전처리', '단어수']] = df['원본'].apply(lambda x: pd.Series(tokenize_and_correct(x)))


,카테고리,원본,직무,토큰화,전처리,단어수
22,지식,"• 국제기구, 양자원조기구, NGO 등의 협력대상국에 대한 개발정책과 지원현황",사업관리,"[국제기구, 양자원조기구, 등의, 협력대상국에, 대한, 개발정책과, 지원현황]",국제기구 양자원 조기구 등의 협력 대상국에 개발정책과 지원 현황,9
23,지식,"• 자국의 공적개발원조 수행방식, 수행기관 업무",사업관리,"[자국의, 공적개발원조, 수행방식, 수행기관, 업무]",자국의 공적개발원조 수행 방식 수행기관 업무,6
24,지식,"• 자국의 공적개발원조 예산, 통계",사업관리,"[자국의, 공적개발원조, 예산, 통계]",자국의 공적개발원조 예산 통계,4
25,지식,• 자국의 공적개발원조 주요 지원 분야,사업관리,"[자국의, 공적개발원조, 주요, 지원, 분야]",자국의 공적개발원조 주요 지원 분야,5
26,지식,"• 자국의 대외정책, 공적개발원조정책, 국별협력전략",사업관리,"[자국의, 대외정책, 공적개발원조정책, 국별협력전략]",자국의 대외정책 공적개발원조 정책 국별 협력전략,6
...,...,...,...,...,...,...
7680,태도,• 유관부서와 정보를 공유하려는 자세,전자기기,"[유관부서와, 정보를, 공유하려는, 자세]",유관 부서와 정보를 공유,4
7681,태도,• 자사제품의 특장점을 설명하려는 자세,전자기기,"[자사제품의, 특장점을, 설명하려는, 자세]",자사 제품의 특장점을 설명,4
7682,태도,• 제품을 상세히 설명하려는 자세,전자기기,"[제품을, 상세히, 설명하려는, 자세]",제품을 상세히 설명,3
7683,태도,• 지속적으로 고객 정보를 수집하려는 자세,전자기기,"[지속적으로, 고객, 정보를, 수집하려는, 자세]",지속적 고객 정보를 수집,4


In [14]:
##단어수의 분포를 나타내주는 그래프 생성
fig = px.histogram(df, x='단어수', nbins=20, title='단어 수 분포 히스토그램')
fig.update_layout(
    xaxis_title='단어 수',
    yaxis_title='빈도 수',
    font=dict(family="Malgun Gothic, Arial, sans-serif")
)
fig.show()


In [15]:
## 전처리 후 필요한 column만 선택 해서 키워드 추출
df = df[["직무","카테고리","원본","전처리"]]
df

,직무,카테고리,원본,전처리
22,사업관리,지식,"• 국제기구, 양자원조기구, NGO 등의 협력대상국에 대한 개발정책과 지원현황",국제기구 양자원 조기구 등의 협력 대상국에 개발정책과 지원 현황
23,사업관리,지식,"• 자국의 공적개발원조 수행방식, 수행기관 업무",자국의 공적개발원조 수행 방식 수행기관 업무
24,사업관리,지식,"• 자국의 공적개발원조 예산, 통계",자국의 공적개발원조 예산 통계
25,사업관리,지식,• 자국의 공적개발원조 주요 지원 분야,자국의 공적개발원조 주요 지원 분야
26,사업관리,지식,"• 자국의 대외정책, 공적개발원조정책, 국별협력전략",자국의 대외정책 공적개발원조 정책 국별 협력전략
...,...,...,...,...
7680,전자기기,태도,• 유관부서와 정보를 공유하려는 자세,유관 부서와 정보를 공유
7681,전자기기,태도,• 자사제품의 특장점을 설명하려는 자세,자사 제품의 특장점을 설명
7682,전자기기,태도,• 제품을 상세히 설명하려는 자세,제품을 상세히 설명
7683,전자기기,태도,• 지속적으로 고객 정보를 수집하려는 자세,지속적 고객 정보를 수집


# 키워드 분류 (TF-IDF, 명사들만 사용, KeyBert, Rake)
TF-IDF -> 단어 빈도수에 따라 점수 계산 <br>
명사들만 사용 -> 데이터를 보다보니 명사들만 추출해도 의미있는 키워드들이 분류 될거 같아서 Okt를 사용해 명사들만 추출<br>
Keybert -> 자연어 처리에서 키워드 추출할 때  많이 쓰이는 라이브러리<br>
Rake -> 문맥이나 구를 파악해 키워드 추출 <br>
데이터 전처리 이후 단어수 분포도를 살펴본 결과, 대부분의 데이터가 단어수 5개이하라 키워드 추출시 3개만 뽑도록 설정


In [16]:
##TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words=None, max_df=0.9, min_df=0.01)
vectorizer.fit(df['전처리'])


def extract_keywords(text, vectorizer, top_n=3):
    tfidf_matrix = vectorizer.transform([text])
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.toarray().flatten()

    top_indices = scores.argsort()[-top_n:][::-1]
    keywords = [feature_names[i] for i in top_indices if scores[i] > 0]

    return " ".join(keywords)


df['TF-IDF'] = df['전처리'].apply(lambda x: extract_keywords(x, vectorizer))

<ipython-input-16-5c09f7bbdfa1>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
##명사만 추출
okt = Okt()
def extract_nouns(text):
    nouns = okt.nouns(text)
    stopwords = ['를', '이', '가', '의', '에', '와', '과']
    filtered_nouns = [noun for noun in nouns if noun not in stopwords]
    return ' '.join(filtered_nouns)


df['명사들'] = df['전처리'].apply(extract_nouns)

In [18]:
## Keybert
kw_model = KeyBERT()
def extract_keywords(text):

    keywords = kw_model.extract_keywords(text, top_n=3)
    keyword_list = [keyword[0] for keyword in keywords]

    # 원본 텍스트에서 키워드 순서대로 정렬
    ordered_keywords = [word for word in text.split() if word in keyword_list]

    return ' '.join(ordered_keywords)

df['키버트'] = df['전처리'].apply(extract_keywords)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



In [19]:
## Rake
rake = Rake()
def rake_keywords(text, top_n=3, max_words=3):

    rake.extract_keywords_from_text(text)
    keywords = rake.get_ranked_phrases_with_scores()


    filtered_keywords = [' '.join(keyword.split()[:max_words]) for _, keyword in keywords]


    return filtered_keywords[:top_n]


df['RAKE'] = df['전처리'].apply(lambda x: rake_keywords(x, top_n=3))

In [20]:
df.head(30)

,직무,카테고리,원본,전처리,TF-IDF,명사들,키버트,RAKE
22,사업관리,지식,"• 국제기구, 양자원조기구, NGO 등의 협력대상국에 대한 개발정책과 지원현황",국제기구 양자원 조기구 등의 협력 대상국에 개발정책과 지원 현황,,국제 기구 자원 기구 등 협력 상국 개발 정책 지원 현황,국제기구 대상국에 개발정책과,[국제기구 양자원 조기구]
23,사업관리,지식,"• 자국의 공적개발원조 수행방식, 수행기관 업무",자국의 공적개발원조 수행 방식 수행기관 업무,,자국 공적 개발원조 수행 방식 수행 기관 업무,공적개발원조 수행 수행기관,[자국의 공적개발원조 수행]
24,사업관리,지식,"• 자국의 공적개발원조 예산, 통계",자국의 공적개발원조 예산 통계,,자국 공적 개발원조 예산 통계,자국의 공적개발원조 통계,[자국의 공적개발원조 예산]
25,사업관리,지식,• 자국의 공적개발원조 주요 지원 분야,자국의 공적개발원조 주요 지원 분야,,자국 공적 개발원조 주요 지원 분야,자국의 공적개발원조 지원,[자국의 공적개발원조 주요]
26,사업관리,지식,"• 자국의 대외정책, 공적개발원조정책, 국별협력전략",자국의 대외정책 공적개발원조 정책 국별 협력전략,,자국 대외 정책 공적 개발원조 정책 국별 협력 전략,자국의 공적개발원조 정책,[자국의 대외정책 공적개발원조]
27,사업관리,지식,• 한국의 협력대상국 지원 실적 및 사례현황,한국의 협력 대상국 지원 실적 사례 현황,,한국 협력 상국 지원 실적 사례 현황,한국의 대상국 현황,[한국의 협력 대상국]
28,사업관리,지식,• 협력대상국의 공적개발원조 담당기관 및 주요 행위자 업무현황,협력 대상국의 공적개발원조 담당기관 주요 행위자 업무 현황,,협력 상국 공적 개발원조 담당 기관 주요 행위자 업무 현황,대상국의 공적개발원조 현황,[협력 대상국의 공적개발원조]
29,사업관리,지식,"• 협력대상국의 법체계, 역사, 정치 경제, 국제관계, 사회정책, 종교와 문화",협력 대상국의 법 체계 역사 정치 경제 국제관계 사회 정책 종교와 문화,,협력 상국 법 체계 역사 정치 경제 국제 관계 사회 정책 종교 문화,대상국의 정책 종교와,[협력 대상국의 법]
30,사업관리,지식,"• 협력대상국의 법체계, 역사, 정치경제, 국제관계, 사회정책, 종교와 문화",협력 대상국의 법 체계 역사 정치경제 국제관계 사회정책 종교와 문화,,협력 상국 법 체계 역사 정치 경제 국제 관계 사회 정책 종교 문화,대상국의 정치경제 종교와,[협력 대상국의 법]
31,사업관리,지식,• 협력대상국의 분야별 사업 운영 사례 및 비즈니스 관행,협력 대상국의 분야별 사업 운영 사례 비즈니스 관행,사업,협력 상국 분야 별 사업 운영 사례 비즈니스 관행,협력 대상국의 운영,[협력 대상국의 분야별]


In [21]:
df.tail(30)

,직무,카테고리,원본,전처리,TF-IDF,명사들,키버트,RAKE
7655,전자기기,기술,• 설문지 작성 능력,설문지 작성,작성,설문지 작성,설문지 작성,[설문지 작성]
7656,전자기기,기술,• 수집정보 가공 능력,수집정보 가공,,수집 정보 가공,수집정보 가공,[수집정보 가공]
7657,전자기기,기술,• 시장정보 분석 능력,시장 정보 분석,정보 시장 분석,시장 정보 분석,시장 정보 분석,[시장 정보 분석]
7658,전자기기,기술,"• 신제품, 신기술에 대한 정보전달 능력",신제품 신기술에 정보 전달,정보,신제품 기술 정보 전달,신제품 신기술에 정보,[신제품 신기술에 정보]
7659,전자기기,기술,• 온오프라인을 통한 정보제공 능력,온오프라인을 통한 정보 제공,정보,오프라인 통한 정보 제공,온오프라인을 통한 정보,[온오프라인을 통한 정보]
7660,전자기기,기술,• 유통채널 파악 능력,유통채널 파악,파악,유통 채널 파악,유통채널 파악,[유통채널 파악]
7661,전자기기,기술,• 자료조사 및 분석 능력,자료 조사 분석,조사 자료 분석,자료 조사 분석,자료 조사 분석,[자료 조사 분석]
7662,전자기기,기술,• 정보 분류 및 통계 능력,정보 분류 통계,정보,정보 분류 통계,정보 분류 통계,[정보 분류 통계]
7663,전자기기,기술,• 정보수집 능력,정보 수집,정보 수집,정보 수집,정보 수집,[정보 수집]
7664,전자기기,기술,• 제품시연 및 교육 능력,제품 시연 교육,교육 제품,제품 시연 교육,제품 시연 교육,[제품 시연 교육]


#데이터 처음 30 그리고 마지막 30개 확인결과 명사들만 추출한 경우가 그래도 가장 문맥적으로 적절하다고 판단해서 이 데이터를 가지고 네트워크 시각화를 진행하고자함.


In [42]:
# 데이터 확인
ab = df[['직무', '카테고리', '명사들']]
ab


,직무,카테고리,명사들
22,사업관리,지식,국제 기구 자원 기구 등 협력 상국 개발 정책 지원 현황
23,사업관리,지식,자국 공적 개발원조 수행 방식 수행 기관 업무
24,사업관리,지식,자국 공적 개발원조 예산 통계
25,사업관리,지식,자국 공적 개발원조 주요 지원 분야
26,사업관리,지식,자국 대외 정책 공적 개발원조 정책 국별 협력 전략
...,...,...,...
7680,전자기기,태도,유관 부서 정보 공유
7681,전자기기,태도,자사 제품 특장점 설명
7682,전자기기,태도,제품 상세 설명
7683,전자기기,태도,지속 고객 정보 수집


In [46]:
# 키워드 출현 빈도 계산 (키워드를 분리하지 않음)
keyword_counts = Counter(ab['명사들'].dropna())

# 키워드 갯수 설정
top_keywords = {k for k, v in keyword_counts.most_common(70)}
ab = ab[ab['명사들'].isin(top_keywords)]

# 네트워크 그래프 생성
G = nx.Graph()

# 직무 노드 추가
for job in ab['직무'].unique():
    G.add_node(job, type='직무')

# 키워드 노드 추가 및 엣지 생성
for _, row in ab.iterrows():
    job = row['직무']
    category = row['카테고리']
    keyword = row['명사들']

    if keyword in top_keywords:
        G.add_node(keyword, type=category)
        G.add_edge(job, keyword)

# 직무 간의 관계 계산
for job1 in ab['직무'].unique():
    for job2 in ab['직무'].unique():
        if job1 != job2:

            common_keywords = set(ab[ab['직무'] == job1]['명사들']).intersection(
                set(ab[ab['직무'] == job2]['명사들']))
            common_count = len(common_keywords)

            # 공통 키워드가 있을 경우 엣지 추가
            if common_count > 0:
                G.add_edge(job1, job2, weight=common_count)

#노트 크기
pos = nx.spring_layout(G, seed=42, k=0.4)
max_size = 30
min_size = 10
max_count = max(keyword_counts.values()) if keyword_counts else 1


edge_x, edge_y = [], []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])


edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# 카테고리 색상 지정
node_x, node_y, node_text, node_color, node_size, node_font_size = [], [], [], [], [], []
category_colors = {'지식': 'blue', '기술': 'green', '태도': 'red', '직무': 'gray'}

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)
    node_type = G.nodes[node].get('type', '직무')
    node_color.append(category_colors.get(node_type, 'gray'))

    if node_type == '직무':
        node_size.append(14)
        node_font_size.append(16)
    else:
        freq = keyword_counts.get(node, 1)
        scaled_size = min_size + (max_size - min_size) * (freq / max_count)
        node_size.append(scaled_size)
        node_font_size.append(10 + (scaled_size / 100))

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=node_text,
    textposition="top center",
    hoverinfo='text',
    marker=dict(
        size=node_size,
        color=node_color,
        opacity=0.8
    ),
    textfont=dict(size=node_font_size)  # 키워드 크기 비례 글씨 크기 조정
)

# 네트워크 그래프 그리기
fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=0),
    title="직무-키워드 네트워크 그래프 (중요도 반영)"
)

fig.show()


#비교 분석 내용 및 인사이트

두 직무 모두 자료 수집 및 분석(통계 분석 포함), 그리고 이를 기반으로 한 커뮤니케이션 및 의사결정이 중요합니다. 또한, 의사소통 능력이 필수적이며, 고객 및 유관 부서와의 협력이 필요합니다.<br>

차이점으로는, 전자기기는 전자제품(설비, 운영, 표준 구격 등)에 대한 지식, 기술, 그리고 고객 대응을 위한 태도가 필요한 반면, 사업관리는 전체적인 프로젝트 진행을 위한 태도, 기술, 능력(프로젝트 관리, 프레젠테이션, 협력, 기획안 작성 등)이 필요합니다.<br>

도출할 수 있는 인사이트로는, 두 직무 모두 데이터 기반의 의사결정이 중요하며, 이를 위해 분석 및 커뮤니케이션 능력이 핵심적입니다. 그러나 차이점으로는, 전자기기 직무는 기술적인 전문성과 고객 서비스가 중요하고, 사업관리 직무는 프로젝트 전체 관리 및 전략적 사고가 필수적이라는 점입니다.
